# SkyNet - A Language Model for Games
Developed by DevArtech (Adam Haile)

Skyjo is a card game which can be played by 2 or more players. The goal of the game is to end the game with the lowest score. The game ends when any player gets over 100 points at the end of any round.
The game is setup as follows:
A deck consists of: 
- 5 cards with a value of -2
- 10 cards with a value of -1
- 15 cards with a value of 0
- Values 1-12, 10 cards of each

To start the game, each player is dealt a total of 12 cards. These cards are not to be seen, and are generally laid in a 4x3 grid in front of the player.
On the first round, each player flips over 2 cards of their choice to start. For the sake of this game, 2 cards are chosen at random. The player with the highest total starts first.
Players then perform the following actions in the given order:
- Choose a deck to draw from. Players can either draw from the deck (which is unknown), or the last discard card (if it exists, this card is known).
- Choose to keep drawn card, or discard it. (Generally, if drawn from the discard, this card is always kept)
- Chose a card on their grid to perform the following action with:
   - If discarded, chose an unknown card of their hand to flip
   - If not discard, chose any card to replace with their drawn card
Once a player has performed this last action, their turn is done and play passes to the next player.
A round ends once any player has revealed all of their cards. When this happens, all players asides from this player have one more turn before the round ends.

When a round ends, players total the values of all of their cards, and add it to their game total score.
If any one player has a lower score than the player who revealed all their cards first, the player who "went out" first has to double their round's score.
This repeats till any player hits 100 points or more at the end of a round, and the player with the lowest score at this time wins.

There is one special rule which is optional to play with, but is included for this model.
If, at any point, a player reveals 3 of the same card in one column, this column is then cleared and removed for the rest of the round. Cards can be of any value though, so while clearing a column of 12s is highly beneficial, clearing a column of -2s is highly detrimental.
The cleared cards are then added to the top of the discard.


The purpose of this model is to play a game of Skyjo. The model is trained by playing against 5 different algorithms, each of which is given different instructions on what to do for a given turn.
- RandomBot: Always choses actions at random. (Random draw, random keep, random placement)
- MiddleBot: Performs actions which are always low (If discard is below 7, take it. If card drawn is below 7, keep it. If drawn card is lower than any visible, replace it, if not, place it in the next unknown index)
- SpeedBot: Performs actions as quickly as possible (Same logic as MiddleBot asides from placement. Unless drawn card can create highly benefical point decreases, given card on board is > 4 and drawn card < given card - swap them, it will place in the next unknown spot.)
- SmartBot: General logic to perform as human as possible (If discard is lower than any given card on the board or low, take it. If drawn card is lower than any card on the board or is medium, keep it. Check all cards on the board and find the largest beneficial point difference to swap with.)
- TripleBot: Performs actions which will create columns of the same card. (If discard is medium or card is on the board, take it. If drawn card is on the board medium, keep it. If card can be placed at a given index and create a clearable column, place it there, otherwise use same logic as Speed for placement.)

These algorithms will be chosen at random to play against the model, with the hope that a diversified set of algorithms will create a model capable to combatting many types of gameplay

This main notebook is meant to be used for the execution of the game, as well as creating and training a SkyNet model.

First, we import packages to create the game and other helper methods

In [7]:
import random
import time
import datetime
import pygame
import game
import bots

pygame 2.1.0 (SDL 2.0.16, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Now, we initialize the pygame before continuing the imports. These are necessary for the display

In [8]:
running = True
display = False

if display:
    pygame.init()
    bounds_x = 1024
    bounds_y = 768
    window = pygame.display.set_mode([bounds_x, bounds_y])
    pygame.display.set_caption("Skyjo")
    from defaults import *
    from cards import Card

Next, we create the game enviornment.

In [9]:
g = game.Skyjo()

Next, we define the type of game. For sake of ease, games can be 2 player at most. Here, we can set whether or not we want two bots to play each other, or a bot and a player. Also included is a helper method to create the bot objects for the game.

In [10]:
def randomBot():
    """
    Sets players at random depending on if one player or two player
    """
    rand = bots.Random()
    mid = bots.Middle()
    speed = bots.Speed()
    smart = bots.Smart()
    triple = bots.Triple()
    if two_bot:
        for i in range(2):
            bot = random.randint(0, 4)
            if bot == 0:
                if i == 0:
                    g.player1 = bots.Random(4, 5, 6, 7, 0, 1, 2, 3)
                else:
                    g.player2 = rand
            elif bot == 1:
                if i == 0:
                    g.player1 = bots.Middle(4, 5, 6, 7, 0, 1, 2, 3)
                else:
                    g.player2 = mid
            elif bot == 2:
                if i == 0:
                    g.player1 = bots.Speed(4, 5, 6, 7, 0, 1, 2, 3)
                else:
                    g.player2 = speed
            elif bot == 3:
                if i == 0:
                    g.player1 = bots.Smart(4, 5, 6, 7, 0, 1, 2, 3)
                else:
                    g.player2 = smart
            else:
                if i == 0:
                    g.player1 = bots.Triple(4, 5, 6, 7, 0, 1, 2, 3)
                else:
                    g.player2 = triple
    else:
        bot = random.randint(0, 4)
        if bot == 0:
            g.player2 = rand
        elif bot == 1:
            g.player2 = mid
        elif bot == 2:
            g.player2 = speed
        elif bot == 3:
            g.player2 = smart
        else:
            g.player2 = triple

Last we create the game loop and run the game

In [11]:
# Comment out timeout for diplay runs
def run_game(running, display, log_game=False, log_console=True):
    g.new_round()

    if log_game:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        game_file = open(f"./recorded_games/{current_time}-{time.time_ns()}_{g.player1.__class__.__name__}_vs_{g.player2.__class__.__name__}.txt", "w")

    if two_bot and log_console:
        print(f"Starting Player: {g.current_player}")
        print(f"{g.player1.__class__.__name__} vs {g.player2.__class__.__name__}")
    if display:
        top_discard = Card(None)
        card_drawn = None

    while running:
        if display:
            player_one_game_cards = []
            player_two_game_cards = []
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            window.fill((95, 141, 186))

            p1 = None
            p2 = None
            if g.get_player_score(1) < g.get_player_score(2):
                p1 = GREEN
                p2 = RED
            elif g.get_player_score(2) < g.get_player_score(1):
                p1 = RED
                p2 = GREEN
            else:
                p1 = YELLOW
                p2 = YELLOW

            text = sky_font.render("Current Player: " +
                                str(g.current_player), True, (255, 255, 255))
            window.blit(text, [bounds_x/2 - 125, bounds_y - 75,
                        text.get_rect().width, text.get_rect().height])

            text = small_font.render(
                "Total Player Score: " + str(g.player_one), True, (255, 255, 255))
            window.blit(text, [bounds_x/20, card_height/2 - 35,
                        text.get_rect().width, text.get_rect().height])

            text = sky_font.render(
                "Player Score: " + str(g.get_player_score(1)), True, p1)
            window.blit(text, [bounds_x/20, card_height/2 - 15,
                        text.get_rect().width, text.get_rect().height])

            text = small_font.render(
                "Total Player Score: " + str(g.player_two), True, (255, 255, 255))
            window.blit(text, [bounds_x/20 + 500, card_height/2 -
                        35, text.get_rect().width, text.get_rect().height])

            text = sky_font.render(
                "Player Score: " + str(g.get_player_score(2)), True, p2)
            window.blit(text, [bounds_x/20 + 500, card_height/2 -
                        15, text.get_rect().width, text.get_rect().height])
            
            i = 0
            
            for y in range(3):
                for x in range(4):
                    if g.player_one_total[i] is not None:
                        new_card = Card(g.player_one_visible[i], g.player_one_total[i], i)
                        new_card.move_to((bounds_x / 10) + (x * 105), (bounds_y / 4) + (y * 200) - 25)
                        new_card.draw(window)
                        player_one_game_cards.append(new_card)

                    if g.player_two_total[i] is not None:
                        new_card = Card(g.player_two_visible[i], g.player_two_total[i], i)
                        new_card.move_to((bounds_x / 10) + (x * 105) + 500, (bounds_y / 4) + (y * 200) - 25)
                        new_card.draw(window)
                        player_two_game_cards.append(new_card)

                    i += 1
            
            deck = Card(None)
            deck.move_to((bounds_x / 10) + 860, bounds_y - 35)
            deck.draw(window)
            
            if len(g.discarded_cards) > 0:
                top_discard = Card(g.discarded_cards[0])
                top_discard.move_to((bounds_x / 10) + 650, bounds_y - 35)
                top_discard.draw(window)

            if g.drawn_card != -3:
                card_drawn = Card(g.drawn_card)
                card_drawn.move_to((bounds_x / 10) + 755, bounds_y - 35)
                card_drawn.draw(window)

            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()

                if g.current_player == 1:
                    for card in player_one_game_cards:
                        if card.get_rect().collidepoint(mouse_pos):
                            if (g.swap_from_deck is True or g.swap_from_discard is True) and g.keep_drawn:
                                g.take_action(card.position, True)
                            elif card.value != card.hidden_value:
                                g.take_action(card.position, False)

                if card_drawn is not None and card_drawn.get_rect().collidepoint(mouse_pos) and g.keep_drawn and g.drawn_card != -3:
                    g.dump_drawn(0)
                    g.drawn_card = -3
                    card_drawn = None

                if deck.get_rect().collidepoint(mouse_pos) and g.swap_from_discard is False and g.drawn_card == -3 and g.keep_drawn:
                    g.swap_from_deck = True
                    g.drawn_card = g.draw_deck()

                if top_discard.get_rect().collidepoint(mouse_pos) and g.swap_from_deck is False and g.drawn_card == -3:
                    g.swap_from_discard = True
                    g.drawn_card = g.draw_discard()

        if g.is_round_over() != 0:
            g.last_swap = True
            if log_game:
                game_file.write("\nLast Draw\n\n")

        if g.current_player == 1 and g.player1 is not None and running:
            choice = g.player1.get_card_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 1 \n<act> Draw \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.card_choice(choice)
            choice = g.player1.get_keep_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 1 \n<act> Keep \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.dump_drawn(choice)
            choice = g.player1.get_placement_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 1 \n<act> Place \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.player_one_placement_choice(choice)
        elif g.current_player == 2 and g.player2 is not None and running:
            choice = g.player2.get_card_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 2 \n<act> Draw \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.card_choice(choice)
            choice = g.player2.get_keep_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 2 \n<act> Keep \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.dump_drawn(choice)
            choice = g.player2.get_placement_choice(g.get_round_state())
            if log_game:
                game_file.write(f"<play> 2 \n<act> Place \n<state> {g.get_round_state()}\n<response> {choice}\n")
            g.player_two_placement_choice(choice)

        if g.last_swap:
            result = g.tally_round_score(running)
            if result == 1:
                running = False
                return f"1 - {g.player1.__class__.__name__}"
            elif result == 2:
                running = False
                return f"2 - {g.player2.__class__.__name__}"
            elif result == 3:
                running = False
                return 3

        if display:
            pygame.display.flip()
            if two_bot:
                time.sleep(1)

    if display: 
        pygame.quit()

    if log_game:
        game_file.close()
    
    g.reset()

# If playing two bots against each other, set two_bot to true, else set to False and specify the type of bot for Player 2 in the initialize_bots method
two_bot = True
if two_bot:
    randomBot()
else:
    g.initialize_bots(bots.Smart())

winner = run_game(running, display)
winner = winner.split(" - ")
print(f"Player {winner[0] if len(winner) > 1 else 'tied'} wins!")

Starting Player: 1
Speed vs Speed
Player 1 wins!


Now that we can play two random bots against each other, we need to create training data

In [12]:
winners = []

import os

for i in range(1000):
    if i % 100 == 0:
        print(f"{len(os.listdir('./recorded_games/'))} games stored")
    randomBot()
    winner = run_game(running, False, True, False)
    winners.append(winner)

random = 0
middle = 0
speed = 0
smart = 0
triple = 0
for winner in winners:
    split = winner.split(" - ")
    if len(split) > 1:
        if split[1] == "Random":
            random += 1
        elif split[1] == "Middle":
            middle += 1
        elif split[1] == "Speed":
            speed += 1
        elif split[1] == "Smart":
            smart += 1
        elif split[1] == "Triple":
            triple += 1

print(f"Random winrate: {(random / len(winners)) * 100}%")
print(f"Middle winrate: {(middle / len(winners)) * 100}%")
print(f"Speed winrate: {(speed / len(winners)) * 100}%")
print(f"Smart winrate: {(smart / len(winners)) * 100}%")
print(f"Triple winrate: {(triple / len(winners)) * 100}%")

0 games stored


100 games stored
200 games stored
300 games stored
400 games stored
500 games stored
600 games stored
700 games stored
800 games stored
900 games stored
Random winrate: 18.9%
Middle winrate: 22.6%
Speed winrate: 19.1%
Smart winrate: 20.4%
Triple winrate: 19.0%
